In [ ]:
import cv2
import numpy as np

In [ ]:
# Load template image
template = cv2.imread('./template-images/face-template-1.jpg', 0)

In [ ]:
# Initialize video capture
cap = cv2.VideoCapture(0)

# Get template dimensions

### Add Code here ###
# (2 points)
h, w = template.shape[:2]
### End Code ###

# Create a Canny edge detector object

### Add code here ###
# ( 2 points)
canny_threshold1 = 50
canny_threshold2 = 200
### End Code ###

# Initialize variables for tracking
prev_top_left = (0, 0)
prev_bottom_right = (1280, 720)
tracking_enabled = True

In [ ]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to grayscale

    ### Add code here ###
    # (5 points)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ### End Code ###
    
    
    # Apply Canny edge detection
     ### Add code here ###
    # (6 points)
    edges = cv2.Canny(gray_frame, canny_threshold1, canny_threshold2)
    ### End Code ###
    
    if tracking_enabled:
        # Define search area based on previous detection
        search_area_top_left = (max(0, prev_top_left[0] - w), max(0, prev_top_left[1] - h))
        search_area_bottom_right = (min(frame.shape[1], prev_bottom_right[0] + w), min(frame.shape[0], prev_bottom_right[1] + h))
        search_area = gray_frame[search_area_top_left[1]:search_area_bottom_right[1], search_area_top_left[0]:search_area_bottom_right[0]]
        
        # Ensure search area size is compatible with template
        if search_area.shape[0] < h or search_area.shape[1] < w:
            tracking_enabled = False
        else:
            # Apply cross correlation in the search area
             ### Add Code here ###
             # (5 points)
            result = cv2.matchTemplate(search_area, template, cv2.TM_CCOEFF_NORMED)
            ### End Code ###
            
            # Find maximum correlation value and its location
            
            ## Add Code here ###
            # (5 points)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            ### End Code ###
            
            # Translate coordinates to global frame
            max_loc = (max_loc[0] + search_area_top_left[0], max_loc[1] + search_area_top_left[1])
            
            # Draw bounding box around the detected object
            ### Add Code here ###
            # (5 points)
            top_left = max_loc
            bottom_right = (top_left[0] + w, top_left[1] + h)
            cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
            ### End Code ###
            
            # Update tracking status and previous location
            prev_top_left = top_left
            prev_bottom_right = bottom_right
    else:
        # Apply cross correlation on the entire frame
        ### Add code here ###
        # (5 points)
        result = cv2.matchTemplate(edges, template, cv2.TM_CCOEFF_NORMED)

        ### End Code###
        
        # Find maximum correlation value and its location
        ### Add Code here ###
        # (5 points)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        ### End Code ###
        
        # Threshold to consider a match
        ### Add Code here ###
        # 1 point
        
        threshold = 0.6

        ### End Code ###
        
        if max_val > threshold:
            # Update tracking status and previous location
            tracking_enabled = True
            prev_top_left = max_loc
            prev_bottom_right = (prev_top_left[0] + w, prev_top_left[1] + h)
            
            # Draw bounding box around the detected object
            ### Add Code Here ###
            # (5 points)
            cv2.rectangle(frame, prev_top_left, prev_bottom_right, (255, 0, 0), 2)
            ### End Code Here ###
    
    # Display the resulting frame
    cv2.imshow('Face Tracking', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()

## Uncomment the following line of code if you want to record face tracking
## and create a video like the one that is embedded in the Homework description.

# out.release()

cv2.destroyAllWindows()
